**Inputs**
* ImageCollection with Images: maximum a posterior (MAP) probabilities of each magnitude-year pair

**Outputs**
* ImageCollection with Images: maximum a posterior (MAP) probabilities of each magnitude-year pair--where magnitude values are thresholds

In [1]:
import ee
ee.Authenticate()

Enter verification code: 4/1AX4XfWg2sIQ5pKgY6cjowSgK7x3kPG1p-OUi8vGqX9omx2Qf69AQj_sL6IQ

Successfully saved authorization token.


In [10]:
ee.Initialize()

In [19]:
WHOLE_GLOBE = ee.Geometry.Rectangle([-179.999, -90, 180, 90], 'EPSG:4326', False)
START_YEAR = 2020
END_YEAR = 2099
hazardname = 'dryspells'
use_greaterthan = True

In [20]:
mapprobs = ee.ImageCollection('users/tedwongwri/dataportal/MAPprobs/{0}'.format(hazardname))

In [24]:
for year in range(START_YEAR, END_YEAR + 1):
    img = mapprobs.filterMetadata('year', 'equals', year).first()
    magnitudes = [float(i.split('_')[1]) for i in img.bandNames().getInfo()]
    year_list = ee.List([])
    for magnitude in magnitudes:
        total_img = ee.Image(0)
        prob_img = ee.Image(0)
        for m1 in magnitudes:
            total_img = total_img.add(img.select('magnitude_{}'.format(str(int(m1)))))
            prob_img = prob_img.add(img.select('magnitude_{}'.format(str(int(m1)))).multiply(((m1 >= magnitude and use_greaterthan) or (m1 <= magnitude and not use_greaterthan)) * 1))
        year_list = year_list.add(prob_img.divide(total_img).set('threshold', magnitude))
    year_img = ee.ImageCollection(year_list).toBands().rename(['threshold_{0}'.format(str(int(i))) for i in magnitudes]).set('year', year)
    task = ee.batch.Export.image.toAsset(**{
        'image': year_img,
        'description': 'thresholdMAPprobs_{0}_{1}'.format(hazardname, str(year)),
        'assetId': 'users/tedwongwri/dataportal/thresholdMAPprobs/{0}/{1}'.format(hazardname, str(year)),
        'region': WHOLE_GLOBE,
        'crs': 'EPSG:4326',
        'dimensions': '1440x720'
    })
    task.start()